In [5]:
import numpy as np
from lr_utils import load_dataset
import h5py
import time

In [6]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
print(train_set_x_orig.shape, train_set_y.shape, test_set_x_orig.shape, test_set_y.shape)
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.
print(train_set_x.shape)

(209, 64, 64, 3) (1, 209) (50, 64, 64, 3) (1, 50)
(12288, 209)


In [7]:
def sigmoid(z):
    a = 1/ (1+ np.exp(-z))
    return a

In [8]:
def RELU(z):
    a = np.maximum(0, z)
    return a

In [9]:
def tanh(z):
    a = np.tanh(z)
    return a

In [10]:
def initialize_zero(dimensions):
    layers = dimensions.shape[0]
    B = np.empty((layers, 1), dtype=object)

    for l in range(layers):
        b = np.zeros((dimensions[l][0], 1))
        B[l, 0] = b
    return B

In [11]:
def initialize_random(dimensions):
    layers = dimensions.shape[0]
    W = np.empty((layers, 1), dtype=object)
    for l in range(layers):
        w = np.random.rand(dimensions[l][1], dimensions[l][0])
        W[l,0] = w
    return W

In [58]:
def HE_initialize(dimensions):
    layers = dimensions.shape[0]
    W = np.empty((layers, 1), dtype=object)
    for l in range(layers):
        #print(dimensions[l][1])
        w = np.random.rand(dimensions[l][1], dimensions[l][0])* np.sqrt(2 /dimensions[l][1])
        W[l,0] = w
    return W

In [59]:
def forward_propagation(W, X, B):
    layers = W.shape[0]
    A = np.empty((layers+ 1, 1), dtype=object)
    A[0][0] = X
    for l in range(layers):
        z = np.dot(W[l][0].T, A[l][0]) + B[l][0]
        if l != layers-1:
            a = RELU(z)
        else:
            a = sigmoid(z)
        A[l+1,0] = a
    return A

In [14]:
#W = HE_initialize(np.array([[3, 12288], [4, 3], [3, 4], [1, 3]]))
#print(W[0])
#B = initialize_zero(np.array([[3, 12288], [4, 3], [3, 4], [1, 3]]))
#A = forward_propagation(W, train_set_x, B)
#print(A[4][0])

In [15]:
def backward_propagation(W, B, A, Y):
    m = Y.shape[1]
    L = W.shape[0] 
    dW = np.empty_like(W)
    dB = np.empty_like(B)
    dZ = None

    A_final = A[L, 0]  # A[4]
    dZ = A_final - Y  # sigmoid derivative for BCE
    dW[L-1, 0] = np.dot(A[L-1, 0], dZ.T) / m  # A3 * dZ4.T
    dB[L-1, 0] = np.sum(dZ, axis=1, keepdims=True) / m

    for l in reversed(range(L - 1)):
        Z = np.dot(W[l, 0].T, A[l, 0]) + B[l, 0]  # recompute Z[l+1]
        dA = np.dot(W[l + 1, 0], dZ)  # backprop through W
        dZ = dA * (Z > 0)  # ReLU derivative

        dW[l, 0] = np.dot(A[l, 0], dZ.T) / m
        dB[l, 0] = np.sum(dZ, axis=1, keepdims=True) / m

    return dW, dB

In [16]:
def calculate_cost(Y, A):
    m = Y.shape[1]
    cost = -1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    return cost


In [17]:
def perdiction(Z):
    for i in range(Z.shape[1]):
        if Z[0][i] > 0.5:
            Z[0][i] = 1
        else:
            Z[0][i] = 0
    return Z
        

In [172]:
def get_layer():
    Layers = int(input("input number of layers:"))
    neurons = []
    for i in range(Layers):
        neurons.append(int(input("input nerouns for " + str(i+1) + "th layer")))
    dim = []
    for i in range(Layers):
        if i == 0:
            dim.append([neurons[0], 12288])
        else:
            dim.append([neurons[i], neurons[i-1]])
        if i == (Layers - 1):
            dim.append([1, neurons[i]])
    dim = np.array(dim)
    return Layers, dim

In [173]:
def model(X, Y, learning_rate = 0.01):
    Layers, dim = get_layer()
    W = HE_initialize(dim)
    B = initialize_zero(dim)
    for i in range(5001):
        A = forward_propagation(W, X, B)
        dW, dB = backward_propagation(W, B, A, Y)
        W -= dW * learning_rate
        B -= dB * learning_rate
        if i%1000 ==0:
            print(calculate_cost(Y, A[-1][0]))
    A = forward_propagation(W, X, B)
    count = 0
    y = perdiction(A[-1][0])
    for i in range(Y.shape[1]):
        if y[0][i] == Y[0][i]:
            count += 1
    accuricy = count/ Y.shape[1] * 100
    print(accuricy)
    return W, B, accuricy

In [164]:
def test_deep(Y, X, W, B):
    count = 0
    A = forward_propagation(W, X, B)
    Z = perdiction(A[-1][0])
    for i in range(Y.shape[1]):
        if Z[0][i] == Y[0][i]:
            count += 1
    accuricy = count/Y.shape[1] * 100
    print(accuricy)
    return accuricy

In [177]:
W, B, acc_train = model(train_set_x, train_set_y)
acc_test = test_deep(test_set_y, test_set_x, W, B)

input number of layers: 1
input nerouns for 1th layer 7


/tmp/ipykernel_18820/2781301467.py:3: RuntimeWarning: divide by zero encountered in log
  cost = -1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
/tmp/ipykernel_18820/2781301467.py:3: RuntimeWarning: invalid value encountered in multiply
  cost = -1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))


nan
0.2867255763893158
0.034481934829264926
0.014998921251779635
0.009820476061251528
0.0076137713476617476
100.0
72.0
